<div style="border: 2px solid #8A9AD0; margin: 1em 0.2em; padding: 0.5em;">

# Generating a single cell matrix using Alevin and combining datasets (bash + R)

by [Julia Jakiela](https://training.galaxyproject.org/hall-of-fame/wee-snufkin/), [Wendi Bacon](https://training.galaxyproject.org/hall-of-fame/nomadscientist/)

CC-BY licensed content from the [Galaxy Training Network](https://training.galaxyproject.org/)

**Objectives**

- I have some single cell FASTQ files I want to analyse. Where do I start?
- How to generate a single cell matrix using command line?

**Objectives**

- Generate a cellxgene matrix for droplet-based single cell sequencing data
- Interpret quality control (QC) plots to make informed decisions on cell thresholds
- Find relevant information in GTF files for the particulars of their study, and include this in data matrix metadata

**Time Estimation: 2H**
</div>


<h1 id="setting-up-the-environment">Setting up the environment</h1>
<p>Alevin is a tool integrated with the <a href="https://salmon.readthedocs.io/en/latest/salmon.html">Salmon software</a>, so first we need to get Salmon. You can install Salmon using conda, but in this tutorial we will show an alternative method - downloading the pre-compiled binaries from the <a href="https://github.com/COMBINE-lab/salmon/releases">releases page</a>. Note that binaries are usually compiled for specific CPU architectures, such as the 64-bit (x86_64) machine release referenced below .</p>


In [ ]:
wget -nv https://github.com/COMBINE-lab/salmon/releases/download/v1.10.0/salmon-1.10.0_linux_x86_64.tar.gz

<p>Once you’ve downloaded a specific binary (here we’re using version 1.10.0), just extract it like so:</p>


In [ ]:
tar -xvzf salmon-1.10.0_linux_x86_64.tar.gz

<blockquote class="details" style="border: 2px solid #ddd; margin: 1em 0.2em">
<div class="box-title details-title" id="details-conda-installation"><button class="gtn-boxify-button details" type="button" aria-controls="details-conda-installation" aria-expanded="true"><i class="fas fa-info-circle" aria-hidden="true" ></i> <span>Details: Conda installation</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>As mentioned, installing salmon using conda is also an option, and you can do it using the following command in the terminal:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">conda install -c bioconda salmon
</code></pre></div>  </div>
<p>However, for this tutorial, it would be easier and quicker to use the downloaded pre-compiled binaries, as shown above.</p>
</blockquote>
<p>We’re going to use Alevin for demonstration purposes, but we do not endorse one method over another.</p>
<h1 id="get-data">Get Data</h1>
<p>We continue working on the same example data - a very small subset of the reads in a mouse dataset of fetal growth restriction {% cite Bacon2018 %} (see the <a href="https://www.ebi.ac.uk/gxa/sc/experiments/E-MTAB-6945/results/tsne">study in Single Cell Expression Atlas</a> and the <a href="https://www.ebi.ac.uk/arrayexpress/experiments/E-MTAB-6945/">project submission</a>). For the purposes of this tutorial, the datasets have been subsampled to only 50k reads (around 1% of the original files). Those are two fastq files - one with transcripts and the another one with cell barcodes. You can download the files by running the code below:</p>


In [ ]:
wget -nv https://zenodo.org/records/10116786/files/transcript_701.fastq
wget -nv https://zenodo.org/records/10116786/files/barcodes_701.fastq

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<p>How to differentiate between the two files if they are just called ‘Read 1’ and ‘Read 2’?</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution"><button class="gtn-boxify-button solution" type="button" aria-controls="solution" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> <span>Solution</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>The file which contains the cell barcodes and UMI is significantly shorter (indeed, 20 bp!) compared to the other file containing longer, transcript read. For ease, we will use explicit file names.</p>
</details>
</blockquote>
<p>Additionally, to map your reads, you will need a transcriptome to align against (a FASTA) as well as the gene information for each transcript (a gtf) file. These files are included in the data import step below. You can also download these for your species of interest <a href="https://www.ensembl.org/info/data/ftp/index.html">from Ensembl</a>.</p>


In [ ]:
wget -c https://zenodo.org/record/4574153/files/Mus_musculus.GRCm38.100.gtf.gff -O GRCm38_gtf.gff
wget -c https://zenodo.org/record/4574153/files/Mus_musculus.GRCm38.cdna.all.fa.fasta -O GRCm38_cdna.fasta

<p>Why do we need FASTA and GTF files?
To generate gene-level quantifications based on transcriptome quantification, Alevin and similar tools require a conversion between transcript and gene identifiers. We can derive a transcript-gene conversion from the gene annotations available in genome resources such as Ensembl. The transcripts in such a list need to match the ones we will use later to build a binary transcriptome index. If you were using spike-ins, you’d need to add these to the transcriptome and the transcript-gene mapping.</p>
<p>We will use the murine reference annotation as retrieved from Ensembl (<em>GRCm38</em> or <em>mm10</em>) in GTF format. This annotation contains gene, exon, transcript and all sorts of other information on the sequences. We will use these to generate the transcript-gene mapping by passing that information to a tool that extracts just the transcript identifiers we need.</p>
<h1 id="generate-a-transcript-to-gene-map-and-filtered-fasta">Generate a transcript to gene map and filtered FASTA</h1>
<p>You can have a look at the Terminal tab again. Has the package <code style="color: inherit">atlas-gene-annotation-manipulation</code> been installed yet? If yes, you can execute the code cell below and while it’s running, I’ll explain all the parameters we set here.</p>


In [ ]:
gtf2featureAnnotation.R -g GRCm38_gtf.gff -c GRCm38_cdna.fasta -d "transcript_id" -t "transcript" -f "transcript_id" -o map -l "transcript_id,gene_id" -r -e filtered_fasta

<p>In essence, <a href="https://github.com/ebi-gene-expression-group/atlas-gene-annotation-manipulation">gtf2featureAnnotation.R script</a> takes a GTF annotation file and creates a table of annotation by feature, optionally filtering a cDNA file supplied at the same time. Therefore the first parameter <code style="color: inherit">-g</code> stands for “gtf-file” and requires a path to a valid GTF file. Then <code style="color: inherit">-c</code> takes a cDNA file for extracting meta info and/or filtering - that’s our FASTA! Where –parse-cdnas (that’s our <code style="color: inherit">-c</code>) is specified, we need to specify, using <code style="color: inherit">-d</code>, which field should be used to compare to identfiers from the FASTA. We set that to “transcript_id” - feel free to inspect the GTF file to explore other attributes. We pass the same value in <code style="color: inherit">-f</code>, meaning first-field, ie. the name of the field to place first in output table. To specify which other fields to retain in the output table, we provide comma-separated list of those fields, and since we’re only interested in transcript to gene map, we put those two names (“transcript_id,gene_id”) into <code style="color: inherit">-l</code>. <code style="color: inherit">-t</code> stands for the feature type to use, and in our case we’re using “transcript”. Guess what <code style="color: inherit">-o</code> is! Indeed, that’s the output annotation table - here we specify the file path of our transcript to gene map. We will also have another output denoted by <code style="color: inherit">-e</code> and that’s the path to a filtered FASTA. Finally, we also put <code style="color: inherit">-r</code> which is there only to suppress header on output. Summarising, output will be a an annotation table, and a FASTA-format cDNAs file with unannotated transcripts removed.</p>
<p>Why filtered FASTA?
Sometimes it’s important that there are no transcripts in a FASTA-format transcriptome that cannot be matched to a transcript/gene mapping. Salmon, for example,  used to produce errors when this mismatch was present. We can synchronise the cDNA file by removing mismatches as we have done above.</p>
<h1 id="generate-a-transcriptome-index">Generate a transcriptome index</h1>
<p>We will use Salmon in mapping-based mode, so first we have to build a Salmon index for our transcriptome. We will run the Salmon indexer as so:</p>


In [ ]:
salmon-latest_linux_x86_64/bin/salmon index -t filtered_fasta -i salmon_index -k 31

<p>Where <code style="color: inherit">-t</code> stands for our filtered FASTA file, and <code style="color: inherit">-i</code> is the output of the mapping-based index. To build it, the function is using an auxiliary k-mer hash over k-mers of length 31. While the mapping algorithms will make use of arbitrarily long matches between the query and reference, the k size selected here will act as the minimum acceptable length for a valid match. Thus, a smaller value of k may slightly improve sensitivity. We find that a k of 31 seems to work well for reads of 75bp or longer, but you might consider a smaller k if you plan to deal with shorter reads. Also, a shorter value of k may improve sensitivity even more when using selective alignment (enabled via the –validateMappings flag). So, if you are seeing a smaller mapping rate than you might expect, consider building the index with a slightly smaller k.</p>
<blockquote class="details" style="border: 2px solid #ddd; margin: 1em 0.2em">
<div class="box-title details-title" id="details-what-is-the-index"><button class="gtn-boxify-button details" type="button" aria-controls="details-what-is-the-index" aria-expanded="true"><i class="fas fa-info-circle" aria-hidden="true" ></i> <span>Details: What is the index?</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>To be able to search a transcriptome quickly, Salmon needs to convert the text (FASTA) format sequences into something it can search quickly, called an ‘index’. The index is in a binary rather than human-readable format, but allows fast lookup by Alevin. Because the types of biological and technical sequences we need to include in the index can vary between experiments, and because we often want to use the most up-to-date reference sequences from Ensembl or NCBI, we can end up re-making the indices quite often.</p>
</blockquote>
<h1 id="use-alevin">Use Alevin</h1>
<p>Time to use Alevin now! Alevin works under the same indexing scheme (as Salmon) for the reference, and consumes the set of FASTA/Q files(s) containing the Cellular Barcode(CB) + Unique Molecule identifier (UMI) in one read file and the read sequence in the other. Given just the transcriptome and the raw read files, Alevin generates a cell-by-gene count matrix (in a fraction of the time compared to other tools).</p>
<blockquote class="details" style="border: 2px solid #ddd; margin: 1em 0.2em">
<div class="box-title details-title" id="details-how-does-alevin-work-in-detail"><button class="gtn-boxify-button details" type="button" aria-controls="details-how-does-alevin-work-in-detail" aria-expanded="true"><i class="fas fa-info-circle" aria-hidden="true" ></i> <span>Details: How does Alevin work in detail?</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<p>Alevin works in two phases. In the first phase it quickly parses the read file containing the CB and UMI information to generate the frequency distribution of all the observed CBs, and creates a lightweight data-structure for fast-look up and correction of the CB. In the second round, Alevin utilizes the read-sequences contained in the files to map the reads to the transcriptome, identify potential PCR/sequencing errors in the UMIs, and performs hybrid de-duplication while accounting for UMI collisions. Finally, a post-abundance estimation CB whitelisting procedure is done and a cell-by-gene count matrix is generated.</p>
</blockquote>
<p>Alevin can be run using the following command:</p>


In [ ]:
salmon-latest_linux_x86_64/bin/salmon alevin -l ISR -1 barcodes_701.fastq -2 transcript_701.fastq --dropseq  -i salmon_index -p 10 -o alevin_output --tgMap map --freqThreshold 3 --keepCBFraction 1 --dumpFeatures

<p>All the required input parameters are described in <a href="https://salmon.readthedocs.io/en/latest/alevin.html">the documentation</a>, but for the ease of use, they are presented below as well:</p>
<blockquote class="details" style="border: 2px solid #ddd; margin: 1em 0.2em">
<div class="box-title details-title" id="details-alevin-input-parameters"><button class="gtn-boxify-button details" type="button" aria-controls="details-alevin-input-parameters" aria-expanded="true"><i class="fas fa-info-circle" aria-hidden="true" ></i> <span>Details: Alevin input parameters</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<ul>
<li>
<p><code style="color: inherit">-l</code>: library type (same as Salmon), we recommend using ISR for both Drop-seq and 10x-v2 chemistry.</p>
</li>
<li>
<p><code style="color: inherit">-1</code>: CB+UMI file(s), alevin requires the path to the FASTQ file containing CB+UMI raw sequences to be given under this command line flag. Alevin also supports parsing of data from multiple files as long as the order is the same as in -2 flag. That’s our barcodes_701.fastq file.</p>
</li>
<li>
<p><code style="color: inherit">-2</code>: Read-sequence file(s), alevin requires the path to the FASTQ file containing raw read-sequences to be given under this command line flag. Alevin also supports parsing of data from multiple files as long as the order is the same as in -1 flag. That’s our transcript_701.fastq file.</p>
</li>
<li>
<p><code style="color: inherit">--dropseq</code> / <code style="color: inherit">--chromium</code> / <code style="color: inherit">--chromiumV3</code>: the protocol, this flag tells the type of single-cell protocol of the input sequencing-library. This is a study using the Drop-seq chemistry, so we specify that in the flag.</p>
</li>
<li>
<p><code style="color: inherit">-i</code>: index, file containing the salmon index of the reference transcriptome, as generated by salmon index command.</p>
</li>
<li>
<p><code style="color: inherit">-p</code>: number of threads, the number of threads which can be used by alevin to perform the quantification, by default alevin utilizes all the available threads in the system, although we recommend using ~10 threads which in our testing gave the best memory-time trade-off.</p>
</li>
<li>
<p><code style="color: inherit">-o</code>: output, path to folder where the output gene-count matrix (along with other meta-data) would be dumped. We simply call it alevin_output</p>
</li>
<li>
<p><code style="color: inherit">--tgMap</code>: transcript to gene map file, a tsv (tab-separated) file — with no header, containing two columns mapping of each transcript present in the reference to the corresponding gene (the first column is a transcript and the second is the corresponding gene). In our case, that’s map_code generated by using gtf2featureAnnotation.R function.</p>
</li>
<li>
<p><code style="color: inherit">--freqThreshold</code> - minimum frequency for a barcode to be considered. We’ve chosen 3 as this will only remove cell barcodes with a frequency of less than 3, a low bar to pass but useful way of avoiding processing a bunch of almost certainly empty barcodes.</p>
</li>
<li>
<p><code style="color: inherit">--keepCBFraction</code> - fraction of cellular barcodes to keep. We’re using 1 to quantify all!</p>
</li>
<li>
<p><code style="color: inherit">--dumpFeatures</code> - if activated, alevin dumps all the features used by the CB classification and their counts at each cell level. It’s generally used in pair with other command line flags.</p>
</li>
</ul>
</blockquote>
<p>We have also added some additional parameters (<code class="language-plaintext highlighter-rouge">--freqThreshold</code>, <code style="color: inherit">--keepCBFraction</code>) and their values are derived from the <a href="{% link topics/single-cell/tutorials/scrna-case_alevin/tutorial.md %}">Alevin Galaxy tutorial</a> after QC to stop Alevin from applying its own thresholds. However, if you’re not sure what value to pick, you can simply allow Alevin to make its own calls on what constitutes empty droplets.</p>
<p>This tool will take a while to run. Alevin produces many file outputs, not all of which we’ll use. You can refer to the <a href="https://salmon.readthedocs.io/en/latest/alevin.html">Alevin documentation</a> if you’re curious what they all are, you can look through all the different files to find information such as the mapping rate, but we’ll just pass the whole output folder directory for downstream analysis.</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title question-title" id="question-1"><i class="far fa-question-circle" aria-hidden="true" ></i> Question</div>
<ol>
<li>Can you find the information what was the mapping rate?</li>
<li>How many transcripts did Alevin find?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em;padding: 0.5em; cursor: pointer;"><summary>👁 View solution</summary>
<div class="box-title solution-title" id="solution-1"><button class="gtn-boxify-button solution" type="button" aria-controls="solution-1" aria-expanded="true"><i class="far fa-eye" aria-hidden="true" ></i> <span>Solution</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<ol>
<li>As mentioned above, in <em>alevin_output</em> folder there will be many different files, including the log files. To check the mapping rate, go to <em>alevin_output</em> -&gt; <em>logs</em> and open <em>salmon_quant</em> file. There you will find not only information about mapping rate, but also many more, calculated at salmon indexing step.</li>
<li>Alevin log can be found in <em>alevin_output</em> -&gt; <em>alevin</em> and the file name is also <em>alevin</em>. You can find many details about the alevin process there, including the number of transcripts found.</li>
</ol>
</details>
</blockquote>
<blockquote class="warning" style="border: 2px solid #de8875; margin: 1em 0.2em">
<div class="box-title warning-title" id="warning-process-stopping"><i class="fas fa-exclamation-triangle" aria-hidden="true" ></i> Warning: Process stopping</div>
<p>The command above will display the log of the process and will say “Analyzed X cells (Y% of all)”. For some reason, running Alevin may sometimes cause problems in Jupyter Notebook and this process will stop and not go to completion. This is the reason why we use hugely subsampled dataset here - bigger ones couldn’t be fully analysed (they worked fine locally though). The dataset used in this tutorial shouldn’t make any issues when you’re using Jupyter notebook through galaxy.eu, however might not work properly on galaxy.org. If you’re accessing Jupyter notebook via galaxy.eu and alevin process stopped, just restart the kernel and that should help.</p>
</blockquote>
<!---
check if we can get alevinQC to work - and paste the info from the first Alevin tutorial?
-->
<h1 id="alevin-output-to-summarizedexperiment">Alevin output to SummarizedExperiment</h1>
<p>Let’s change gear a little bit. We’ve done the work in bash, and now we’re switching to R to complete the processing. To do so, you have to change Kernel to R (either click on <code style="color: inherit">Kernel</code> -&gt; <code style="color: inherit">Change Kernel...</code> in the upper left corner of your JupyterLab or click on the displayed current kernel in the upper right corner and change it).</p>
<figure id="figure-1" style="max-width: 90%;"><img src="...
EBERAREQEREBERAREQf/2Q==
"" alt="Figure showing the JupyterLab interface with an arrow pointing to the left corner, showing the option 'Kernel' -&gt; 'Change Kernel...' and another arrow pointing to the right corner, showing the icon of the current kernel. The pop-up window asks which kernel should be chosen instead." width="1760" height="580" loading="lazy" /><a target="_blank" href="../../images/scrna-pre-processing/switch_kernel.jpg" rel="noopener noreferrer"><small>Open image in new tab</small></a><br /><br /><figcaption><span class="figcaption-prefix"><strong>Figure 1</strong>:</span> Two ways of switching kernel.</figcaption></figure>
<p>Now load the library that we have previously installed in terminal:</p>


In [ ]:
library(tximeta)

<p>The <a href="https://bioconductor.org/packages/devel/bioc/vignettes/tximeta/inst/doc/tximeta.html">tximeta package</a> created by {% cite Love2020 %} is used for import of transcript-level quantification data into R/Bioconductor and requires that the entire output of alevin is present and unmodified.</p>
<p>In the <em>alevin_output</em> -&gt; <em>alevin</em> folder you can find the following files:</p>
<ul>
<li><em>quants_mat.gz</em>- Compressed count matrix</li>
<li><em>quants_mat_rows.txt</em>- Row Index (CB-ids) of the matrix.</li>
<li><em>quants_mat_cols.txt</em> - Column Header (Gene-ids) of the matrix.</li>
<li><em>quants_tier_mat.gz</em> – Tier categorization of the matrix.</li>
</ul>
<p>We will only focus on <em>quants_mat.gz</em> though. First, let’s specify the path to that file:</p>


In [ ]:
path <- 'alevin_output/alevin/quants_mat.gz'

<p>We will specify the following arguments when running <em>tximeta</em>:</p>
<ul>
<li>‘coldata’ a data.frame with at least two columns:
<ul>
<li>files - character, paths of quantification files</li>
<li>names - character, sample names</li>
</ul>
</li>
<li>‘type’ - what quantifier was used (can be ‘salomon’, ‘alevin’, etc.)</li>
</ul>
<p>With that we can create a dataframe and pass it to tximeta to create SummarizedExperiment object.</p>


In [ ]:
coldata <- data.frame(files = path, names="sample701")
alevin_se <- tximeta(coldata, type = "alevin")

<p>Inspect the created object:</p>


In [ ]:
alevin_se

<p>As you can see, <em>rowData names</em> and <em>colData names</em> are still empty. Before we add some metadata,  we will first identify barcodes that correspond to non-empty droplets.</p>
<h1 id="identify-barcodes-that-correspond-to-non-empty-droplets">Identify barcodes that correspond to non-empty droplets</h1>
<p>Some sub-populations of small cells may not be distinguished from empty droplets based purely on counts by barcode. Some libraries produce multiple ‘knees’ (see the <a href="{% link topics/single-cell/tutorials/scrna-case_alevin/tutorial.md %}#basic-qc">Alevin Galaxy tutorial</a> for multiple sub-populations. The <code style="color: inherit">emptyDrops</code> method ({% cite Lun2019 %}) has become a popular way of dealing with this. <code style="color: inherit">emptyDrops</code> still retains barcodes with very high counts, but also adds in barcodes that can be statistically distinguished from the ambient profiles, even if total counts are similar.</p>


In [ ]:
library(DropletUtils)               # load the library and required packages

<p>emptyDrops takes multiple arguments that you can read about in the <a href="https://rdrr.io/github/MarioniLab/DropletUtils/man/emptyDrops.html">documentation</a>. However, in this case, we will only specify the following arguments:</p>
<blockquote class="details" style="border: 2px solid #ddd; margin: 1em 0.2em">
<div class="box-title details-title" id="details-emptydrops-input-parameters"><button class="gtn-boxify-button details" type="button" aria-controls="details-emptydrops-input-parameters" aria-expanded="true"><i class="fas fa-info-circle" aria-hidden="true" ></i> <span>Details: emptyDrops input parameters</span><span class="fold-unfold fa fa-minus-square"></span></button></div>
<ul>
<li><code style="color: inherit">m</code> -	A numeric matrix-like object - usually a dgTMatrix or dgCMatrix - containing droplet data prior to any filtering or cell calling. Columns represent barcoded droplets, rows represent genes.</li>
<li><code style="color: inherit">lower</code> - A numeric scalar specifying the lower bound on the total UMI count, at or below which all barcodes are assumed to correspond to empty droplets.</li>
<li><code style="color: inherit">niters</code> - An integer scalar specifying the number of iterations to use for the Monte Carlo p-value calculations.</li>
<li><code style="color: inherit">retain</code> - A numeric scalar specifying the threshold for the total UMI count above which all barcodes are assumed to contain cells.</li>
</ul>
</blockquote>
<p>Let’s then extract the matrix from our <code style="color: inherit">alevin_se</code> object. It’s stored in <em>assays</em> -&gt; <em>counts</em>.</p>


In [ ]:
matrix_alevin <- assays(alevin_se)$counts

<p>And now run emptyDrops:</p>


In [ ]:
# Identify likely cell-containing droplets
out <- emptyDrops(matrix_alevin, lower = 100, niters = 1000, retain = 20)
out

<p>We also correct for multiple testing by controlling the false discovery rate (FDR) using the Benjamini-Hochberg (BH) method ({% cite Benjamini1995 %}). Putative cells are defined as those barcodes that have significantly poor fits to the ambient model at a specified FDR threshold. Here, we will use an FDR threshold of 0.01. This means that the expected proportion of empty droplets in the set of retained barcodes is no greater than 1%, which we consider to be acceptably low for downstream analyses.</p>


In [ ]:
is.cell <- out$FDR <= 0.01
sum(is.cell, na.rm=TRUE)                              # check how many cells left after filtering

<p>We got rid of the background droplets containing no cells, so now we will filter the matrix that we passed on to emptyDrops, so that it corresponds to the remaining cells.</p>


In [ ]:
emptied_matrix <- matrix_alevin[,which(is.cell),drop=FALSE]          # filter the matrix

<p>From here, we can move on to adding metadata and we’ll return to <code style="color: inherit">emptied_matrix</code> soon.</p>
<h1 id="adding-cell-metadata">Adding cell metadata</h1>
<p>The cells barcodes are stored in <em>colnames</em>. Let’s exctract them into a separate object:</p>


In [ ]:
barcode <- colnames(alevin_se)

<p>Now, we can simply add those barcodes into <em>colData names</em> where we will keep the cell metadata. To do this, we will create a column called <code style="color: inherit">barcode</code> in <em>colData</em> and pass the stored values into there.</p>


In [ ]:
colData(alevin_se)$barcode <- barcode
colData(alevin_se)

<p>That’s only cell barcodes for now! However, after running <em>emptyDrops</em>, we generated lots of cell information that is currently stored in <code style="color: inherit">out</code> object (Total, LogProb, PValue, Limited, FDR). Let’s add those values to cell metadata! Since we already have <em>barcodes</em> in there, we will simply bind the emptyDrops output to the existing dataframe:</p>


In [ ]:
colData(alevin_se) <- cbind(colData(alevin_se),out)
colData(alevin_se)

<p>As you can see, the new columns were appended successfully and now the dataframe has 6 columns.</p>
<p>If you have a look at the Experimental Design from that study, you might notice that there is actually more information about the cells. The most important for us would be batch, genotype and sex, summarised in the small table below.</p>
<table>
<thead>
<tr>
<th>Index</th>
<th>Batch</th>
<th>Genotype</th>
<th>Sex</th>
</tr>
</thead>
<tbody>
<tr>
<td>N701</td>
<td>0</td>
<td>wildtype</td>
<td>male</td>
</tr>
<tr>
<td>N702</td>
<td>1</td>
<td>knockout</td>
<td>male</td>
</tr>
<tr>
<td>N703</td>
<td>2</td>
<td>knockout</td>
<td>female</td>
</tr>
<tr>
<td>N704</td>
<td>3</td>
<td>wildtype</td>
<td>male</td>
</tr>
<tr>
<td>N705</td>
<td>4</td>
<td>wildtype</td>
<td>male</td>
</tr>
<tr>
<td>N706</td>
<td>5</td>
<td>wildtype</td>
<td>male</td>
</tr>
<tr>
<td>N707</td>
<td>6</td>
<td>knockout</td>
<td>male</td>
</tr>
</tbody>
</table>
<p>We are currently analysing sample N701, so let’s add its information from the table.</p>
<h2 id="batch">Batch</h2>
<p>We will label batch as an integer - “0” for sample N701, “1” for N702 etc. The way to do it is creating a list with zeros of the length corresponding to the number of cells that we have in our SummarizedExperiment object, like so:</p>


In [ ]:
batch <- rep("0", length(colnames(alevin_se)))

<p>And now create a batch slot in the <em>colData names</em> and append the <code style="color: inherit">batch</code> list in the same way as we did with barcodes:</p>


In [ ]:
colData(alevin_se)$batch <- batch
colData(alevin_se)

<p>A new column appeared, full of zeros - as expected!</p>
<h2 id="genotype">Genotype</h2>
<p>It’s all the same for genotype, but instead creating a list with zeros, we’ll create a list with string “wildtype” and append it into genotype slot:</p>


In [ ]:
genotype <- rep("wildtype", length(colnames(alevin_se)))
colData(alevin_se)$genotype <- genotype

<h2 id="sex">Sex</h2>
<p>You already know what to do, right? A list with string “male” and then adding it into a new slot!</p>


In [ ]:
sex <- rep("male", length(colnames(alevin_se)))
colData(alevin_se)$sex <- sex

<p>Check if all looks fine:</p>


In [ ]:
colData(alevin_se)

<p>3 new columns appeared with the information that we’ve just added - perfect! You can add any information you need in this way, as long as it’s the same for all the cells from one sample.</p>
<h1 id="adding-gene-metadata">Adding gene metadata</h1>
<p>The genes IDs are stored in <em>rownames</em>. Let’s exctract them into a separate object:</p>


In [ ]:
gene_ID <- rownames(alevin_se)

<p>Analogically, we will add those genes IDs into <em>rowData names</em> which stores gene metadata. To do this, we will create a column called <code style="color: inherit">gene_ID</code> in <em>rowData</em> and pass the stored values into there.</p>


In [ ]:
rowData(alevin_se)$gene_ID <- gene_ID

<h2 id="adding-genes-symbols-based-on-their-ids">Adding genes symbols based on their IDs</h2>
<p>Since gene symbols are much more informative than only gene IDs, we will add them to our metadata. We will base this annotation on Ensembl - the genome database – with the use of the library BioMart. We will use the archive Genome assembly GRCm38 to get the gene names. Please note that the updated version (GRCm39) is available, but some of the gene IDs are not in that EnsEMBL database. The code below is written in a way that it will work for the updated dataset too, but will produce ‘NA’ where the corresponding gene name couldn’t be found.</p>


In [ ]:
# get relevant gene names
library("biomaRt")                                      # load the BioMart library
ensembl.ids <- gene_ID
mart <- useEnsembl(biomart = "ENSEMBL_MART_ENSEMBL")    # connect to a specified BioMart database and dataset hosted by Ensembl
ensembl_m = useMart("ensembl", dataset="mmusculus_gene_ensembl", version=100)

# The line above connects to a specified BioMart database and dataset within this database.
# In our case we choose the mus musculus database and to get the desired Genome assembly GRCm38,
# we specify the host with this archive. If you want to use the most recent version of the dataset, just run:
# ensembl_m = useMart("ensembl", dataset="mmusculus_gene_ensembl")

<blockquote class="warning" style="border: 2px solid #de8875; margin: 1em 0.2em">
<div class="box-title warning-title" id="warning-ensembl-connection-problems"><i class="fas fa-exclamation-triangle" aria-hidden="true" ></i> Warning: Ensembl connection problems</div>
<p>Sometimes you may encounter some connection issues with Ensembl. To improve performance Ensembl provides several mirrors of their site distributed around the globe. When you use the default settings for useEnsembl() your queries will be directed to your closest mirror geographically. In theory this should give you the best performance, however this is not always the case in practice. For example, if the nearest mirror is experiencing many queries from other users it may perform poorly for you. In such cases, the other mirrors should be chosen automatically.</p>
</blockquote>


In [ ]:
genes <- getBM(attributes=c('ensembl_gene_id','external_gene_name'),
               filters = 'ensembl_gene_id',
               values = ensembl.ids,
               mart = ensembl_m)

# The line above retrieves the specified attributes from the connected BioMart database;
# 'ensembl_gene_id' are genes IDs,
# 'external_gene_name' are the genes symbols that we want to get for our values stored in ‘ensembl.ids’.

In [ ]:
# see the resulting data
head(genes)

In [ ]:
# replace IDs for gene names
gene_names <- ensembl.ids
count = 1
for (geneID in gene_names)
{
 index <- which(genes==geneID)    # finds an index of geneID in the genes object created by getBM()
 if (length(index)==0)            # condition in case if there is no corresponding gene name in the chosen dataset
  {
    gene_names[count] <- 'NA'
  }
  else
  {
    gene_names[count] <- genes$external_gene_name[index] 	# replaces gene ID by the corresponding gene name based on the found geneID’s index
  }
 count = count + 1                # increased count so that every element in gene_names is replaced
}

In [ ]:
# add the gene names into rowData in a new column gene_name
rowData(alevin_se)$gene_name <- gene_names

In [ ]:
# see the changes
rowData(alevin_se)

<p>If you are working on your own data and it’s not mouse data, you can check available datasets for other species and just use relevant dataset in <code style="color: inherit">useMart()</code> function.</p>


In [ ]:
listDatasets(mart)                # available datasets

<h2 id="flag-mitochondrial-genes">Flag mitochondrial genes</h2>
<p>We can also flag mitochondrial genes. Usually those are the genes whose name starts with ‘mt-‘ or ‘MT-‘. Therefore, we will store those characters in <code style="color: inherit">mito_genes_names</code> and then use <em>grepl()</em> to find those genes stored in <em>gene_name</em> slot.</p>


In [ ]:
mito_genes_names <- '^mt-|^MT-'                                    # how mitochondrial gene names can start
mito <- grepl(mito_genes_names, rowData(alevin_se)$gene_name)      # find mito genes
mito                                                               # see the resulting boolean list

<p>Now we can add another slot in <em>rowData</em> and call it <em>mito</em> that will keep boolean values (true/false) to indicate which genes are mitochondrial.</p>


In [ ]:
rowData(alevin_se)$mito <- mito
rowData(alevin_se)

<h1 id="subsetting-the-object">Subsetting the object</h1>
<p>Let’s go back to the <code style="color: inherit">emptied_matrix</code> object. Do you remember how many cells were left after filtering? We can check that by looking at the matrix’ dimensions:</p>


In [ ]:
dim(matrix_alevin)                                                  # check the dimension of the unfiltered matrix
dim(emptied_matrix)                                                 # check the dimension of the filtered matrix

<p>We’ve gone from 3608 to 35 cells. We’ve filtered the matrix, but not our SummarizedExperiment. We can subset <code style="color: inherit">alevin_se</code> based on the cells that were left after filtering. We will store them in a separate list, as we did with the barcodes:</p>


In [ ]:
retained_cells <- colnames(emptied_matrix)
retained_cells

<p>And now we can subset our SummarizedExperiment based on the barcodes that are in the <code style="color: inherit">retained_cells</code> list:</p>


In [ ]:
alevin_subset <- alevin_se[, colData(alevin_se)$barcode %in% retained_cells]
alevin_701 <- alevin_subset
alevin_701

<p>And that’s our subset, ready for downstream analysis!</p>
<h1 id="more-datasets">More datasets</h1>
<p>We’ve done the analysis for one sample. But there are 7 samples in this experiment and it would be very handy to have all the information in one place. Therefore, you would need to repeat all the steps for the subsequent samples (that’s when you’ll appreciate wrapped tools and automation in Galaxy workflows!). To make your life easier, we will show you how to combine the datasets on smaller scale. Also, to save you some time, we’ve already run Alevin on sample 702 (also subsampled to 50k reads). Let’s quickly repeat the steps we performed in R to complete the analysis of sample 702 in the same way as we did with 701.</p>
<p>But first, we have to save the results of our hard work on sample 701!</p>
<h2 id="saving-sample-701-data">Saving sample 701 data</h2>
<p>Saving files is quite straightforward. Just specify which object you want to save and how you want the file to be named. Don’t forget the extension!</p>


In [ ]:
save(alevin_701, file = "alevin_701.rdata")

<p>You will see the new file in the panel on the left.</p>
<h2 id="analysis-of-sample-702">Analysis of sample 702</h2>
<p>Normally, at this point you would switch kernel to bash to run alevin, and then back to R to complete the analysis of another sample. Here, we are providing you with the alevin output for the next sample, but to give you some practise in switching kernels and saving data, we will use bash to unzip the folder with that output data.</p>
<blockquote class="warning" style="border: 2px solid #de8875; margin: 1em 0.2em">
<div class="box-title warning-title" id="warning-switching-kernels-amp-losing-variables"><i class="fas fa-exclamation-triangle" aria-hidden="true" ></i> Warning: Switching kernels &amp; losing variables</div>
<p>Be aware that every time when you switch kernel, you will lose variables you store in the objects that you’ve created, unless you save them. Therefore, if you want to switch from R to bash, make sure you save your R objects! You can then load them anytime.</p>
</blockquote>
<p>Let’s <strong>switch the kernel back to bash</strong> and run the following code to unzip the alevin output for sample 702:</p>


In [ ]:
# we're in bash again!
wget https://zenodo.org/records/10116786/files/alevin_output_702.zip

In [ ]:
unzip alevin_output_702.zip

<p>The files are there! Now <strong>back to R - switch kernel again</strong>.</p>
<p>Above we described all the steps done in R and explained what each bit of code does. Below all those steps are in one block of code, so read carefully and make sure you understand everything!</p>


In [ ]:
# we're in R now!

## load libraries again ##
library(tximeta)
library(DropletUtils)
library(biomaRt)

## running tximeta ##
path2 <- 'alevin_output_702/alevin/quants_mat.gz'                                                    # path to alevin output for N702
alevin2 <- tximeta(coldata = data.frame(files = path2, names = "sample702"), type = "alevin")        # create SummarizedExperiment from Alevin output

## running emptyDrops ##
matrix_alevin2 <- assays(alevin2)$counts                                                             # extract matrix from SummarizedExperiment
out2 <- emptyDrops(matrix_alevin2, lower = 100, niters = 1000, retain = 20)                          # apply emptyDrops
is.cell2 <- out2$FDR <= 0.01                                                                         # apply FDR threshold
emptied_matrix2 <- matrix_alevin2[,which(is.cell2),drop=FALSE]                                       # subset the matrix to the cell-containing droplets

## adding cell metadata ##
barcode2 <- colnames(alevin2)
colData(alevin2)$barcode <- barcode2                                             # add barcodes

colData(alevin2) <- cbind(colData(alevin2),out2)                                 # add emptyDrops info

batch2 <- rep("1", length(colnames(alevin2)))
colData(alevin2)$batch <- batch2                                                # add batch info

genotype2 <- rep("wildtype", length(colnames(alevin2)))
colData(alevin2)$genotype <- genotype2                                          # add genotype info

sex2 <- rep("male", length(colnames(alevin2)))
colData(alevin2)$sex <- sex2                                                    # add sex info

## adding gene metadata ##
gene_ID2 <- rownames(alevin2)
rowData(alevin2)$gene_ID <- gene_ID2

# get relevant gene names
ensembl.ids2 <- gene_ID2
mart <- useEnsembl(biomart = "ENSEMBL_MART_ENSEMBL")    # connect to a specified BioMart database and dataset hosted by Ensembl
ensembl_m2 = useMart("ensembl", dataset="mmusculus_gene_ensembl", version=100)

genes2 <- getBM(attributes=c('ensembl_gene_id','external_gene_name'),
               filters = 'ensembl_gene_id',
               values = ensembl.ids2,
               mart = ensembl_m2)

# replace IDs for gene names
gene_names2 <- ensembl.ids2
count = 1
for (geneID in gene_names2)
{
 index <- which(genes2==geneID)    # finds an index of geneID in the genes object created by getBM()
 if (length(index)==0)            # condition in case if there is no corresponding gene name in the chosen dataset
  {
    gene_names2[count] <- 'NA'
  }
  else
  {
    gene_names2[count] <- genes2$external_gene_name[index] 	# replaces gene ID by the corresponding gene name based on the found geneID’s index
  }
 count = count + 1                # increased count so that every element in gene_names is replaced
}

rowData(alevin2)$gene_name <- gene_names2                              # add gene names to gene metadata

mito_genes_names <- '^mt-|^MT-'                                        # how mitochondrial gene names can start
mito2 <- grepl(mito_genes_names, rowData(alevin2)$gene_name)           # find mito genes
rowData(alevin2)$mito <- mito2                                         # add mitochondrial information to gene metadata

## create a subset of filtered object ##
retained_cells2 <- colnames(emptied_matrix2)
alevin_subset2 <- alevin2[, colData(alevin2)$barcode %in% retained_cells2]

alevin_702 <- alevin_subset2
alevin_702

<p>Alright, another sample pre-processed!</p>
<h1 id="combining-datasets">Combining datasets</h1>
<p>Pre-processed sample 702 is there, but we still need to load sample 701 that we saved before switching kernels. It’s equally easy as saving the object:</p>


In [ ]:
load("alevin_701.rdata")

<p>Check if it was loaded ok:</p>


In [ ]:
alevin_701

<p>Now we can combine those two objects into one using one simple command:</p>


In [ ]:
alevin_combined <- cbind(alevin_701, alevin_702)
alevin_combined

<p>If you have more samples, just append them in the same way. We won’t process another sample here, but pretending that we have third sample, we would combine it like this:</p>


In [ ]:
alevin_subset3 <- alevin_702                        # copy dataset for demonstration purposes
alevin_combined_demo <- cbind(alevin_combined, alevin_subset3)
alevin_combined_demo

<p>You get the point, right? It’s important though that the rowData names and colData names are the same in each sample.</p>
<h1 id="saving-and-exporting-the-files">Saving and exporting the files</h1>
<p>It is generally more common to use SingleCellExperiment format rather than SummarizedExperiment. The conversion is quick and easy, and goes like this:</p>


In [ ]:
library(SingleCellExperiment)                                 # might need to load this library
alevin_sce <- as(alevin_combined, "SingleCellExperiment")
alevin_sce

<p>As you can see, all the embeddings have been successfully transfered during this conversion and believe me, sce object will be more useful for you!</p>
<p>You’ve already learned how to save and load objects in Jupyter notebook, let’s then save the SCE file:</p>


In [ ]:
save(alevin_sce, file = "alevin_sce.rdata")

<p>The last thing that might be useful is exporting the files into your Galaxy history. To do it… guess what! Yes - <strong>switching kernels again</strong>! But this time we choose <strong>Python 3</strong> kernel and run the following command:</p>


In [ ]:
# that's Python now!
put("alevin_sce.rdata")

<h1 id="conclusion">Conclusion</h1>
<p>Well done! In this tutorial we have:</p>
<ul>
<li>examined raw read data, annotations and necessary input files for quantification</li>
<li>created an index in Salmon and run Alevin</li>
<li>identified barcodes that correspond to non-empty droplets</li>
<li>added gene and cell metadata</li>
<li>applied the necessary conversion to pass these data to downstream processes.</li>
</ul>
<p>As you might now appreciate, some tasks are much quicker and easier when run in the code, but the reproducibility and automation of Galaxy workflows is a huge advantage that helps in dealing with many samples more quickly and efficiently.</p>


# Key Points

- Create a SCE object from FASTQ files, including relevant gene and cell metadata, and do it all in Jupyter Notebook!

# Congratulations on successfully completing this tutorial!

Please [fill out the feedback on the GTN website](https://training.galaxyproject.org/training-material/topics/single-cell/tutorials/alevin-commandline/tutorial.html#feedback) and check there for further resources!
